In [2]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    os.environ['CUDA_VISIBLE_DEVICES'] = \
        os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0]
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


4,5
4
Sun Nov 19 15:04:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   31C    P0    64W / 300W |  28351MiB / 32510MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| 

In [3]:
import os
import json
import argparse
import logging
import random
import numpy as np
import torch
import datasets
from alpaca_eval import evaluate as alpaca_farm_evaluate
from eval.utils import query_openai_chat_model, query_openai_model, generate_completions, dynamic_import_function, load_hf_lm_and_tokenizer

[2023-11-19 15:04:15,194] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [13]:
parser = argparse.ArgumentParser()
parser.add_argument("--reference_path", type=str, default="data/eval/alpaca_farm/davinci_003_outputs_2048_token.json", help="Path to the reference outputs. Alpaca_eval leaderboard use davinci_003 to generate the reference outputs, but they limit the max_tokens to 300. Here regenerated reference outputs with max_tokens=2048.")
parser.add_argument("--save_dir", type=str, default="results/alpaca_farm")
parser.add_argument("--model_name_or_path", type=str, default=None, help="If specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="If specified, we will load the tokenizer from here.")
parser.add_argument("--use_slow_tokenizer", action="store_true", help="If given, we will use the slow tokenizer.")
parser.add_argument("--openai_engine", type=str, default=None, help="If specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="Batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="Load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, we will use the chat format for the prompts.")
parser.add_argument("--chat_formatting_function", type=str, default="eval.templates.create_prompt_with_tulu_chat_format", help="The function to use to create the chat format. This function will be dynamically imported. Please see examples in `eval/templates.py`.")
parser.add_argument("--use_vllm", action="store_true", help="If given, we will use vLLM to generate the predictions - much faster.")
parser.add_argument("--annotators_config", type=str, default="alpaca_eval_gpt4_0314")
parser.add_argument("--max_num_examples", type=int, default=None, help="maximum number of examples to evaluate.")


model_name_or_path = '../results/baselines/huggyllama/llama-7b'
model_name_or_path = '../results/ft1_ep=1/llama-7b_tuluv1m'
model_name_or_path = '../results/oi5_ultrachat:mistral-7b/mistral-7b_ultrachat200k_score=log:prob:neg_pace=prune:size=50000:ep=3'
model_name_or_path = '../results/ft1_ep=2/mistral-7b_ultrachat200k_beforesplitlongconv'
model_name_or_path = '../results/oi5_ultrachat15:mistral-7b/mistral-7b_ultrachat15_score=semdedup:cl=kmeansfaisscd:md=mpnet:dist=cd:emb=text+embedding:nc=200_pace=prune:size=100000:ep=2'

save_dir = os.path.join(model_name_or_path, 'eval/alpacafarm_jpt/')

# eval_batch_size <=5 without oom for llama-7b gen 2048 new tokens

#     --use_chat_format \
#     --max_num_examples 50 \
#     --max_num_examples 5 \
cmd = f"""
    --reference_path alpaca_eval_data \
    --model_name_or_path {model_name_or_path} \
    --save_dir {save_dir} \
    --eval_batch_size 5 \
    --use_chat_format \
    --annotators_config chatgpt \
"""

args = jpt_parse_args(parser, cmd)
args

Namespace(reference_path='alpaca_eval_data', save_dir='../results/oi5_ultrachat15:mistral-7b/mistral-7b_ultrachat15_score=semdedup:cl=kmeansfaisscd:md=mpnet:dist=cd:emb=text+embedding:nc=200_pace=prune:size=100000:ep=2/eval/alpacafarm_jpt/', model_name_or_path='../results/oi5_ultrachat15:mistral-7b/mistral-7b_ultrachat15_score=semdedup:cl=kmeansfaisscd:md=mpnet:dist=cd:emb=text+embedding:nc=200_pace=prune:size=100000:ep=2', tokenizer_name_or_path=None, use_slow_tokenizer=False, openai_engine=None, eval_batch_size=5, load_in_8bit=False, gptq=False, use_chat_format=True, chat_formatting_function='eval.templates.create_prompt_with_tulu_chat_format', use_vllm=False, annotators_config='chatgpt', max_num_examples=None)

In [23]:

random.seed(42)
os.makedirs(args.save_dir, exist_ok=True)

if args.reference_path != 'alpaca_eval_data':
    raise NotImplementedError('Only support alpaca_eval_data for now. Need to work with `max_num_examples` in case I want to use other reference generation.')

logging.info("loading data and model...")
alpaca_eval_data = datasets.load_dataset("tatsu-lab/alpaca_farm", "alpaca_farm_evaluation")["eval"]

if args.max_num_examples and len(alpaca_eval_data) > args.max_num_examples:
    inds = random.sample(range(len(alpaca_eval_data)), args.max_num_examples)
    alpaca_eval_data = alpaca_eval_data.select(inds)

prompts = []
chat_formatting_function = dynamic_import_function(args.chat_formatting_function) if args.use_chat_format else None
for example in alpaca_eval_data:
    prompt = example["instruction"] + "\n\n" + example["input"] if example["input"] != "" else example["instruction"]
    if args.use_chat_format:
        messages = [{"role": "user", "content": prompt}]
        prompt = chat_formatting_function(messages, add_bos=False)
    prompts.append(prompt)


INFO:root:loading data and model...


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
alpaca_eval_data.filter(lambda x: 'ophthalmologist' in x['instruction'])

Dataset({
    features: ['instruction', 'input', 'output', 'generator', 'sample_mode', 'dataset', 'datasplit'],
    num_rows: 0
})

In [19]:
model, tokenizer = load_hf_lm_and_tokenizer(
    model_name_or_path=args.model_name_or_path,
    tokenizer_name_or_path=args.tokenizer_name_or_path if args.tokenizer_name_or_path is not None else args.model_name_or_path,
    load_in_8bit=args.load_in_8bit,
    device_map="balanced_low_0" if torch.cuda.device_count() > 1 else "auto",
    gptq_model=args.gptq,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [109]:
# args.annotators_config

import pandas as pd
from alpaca_eval import annotators


class MyAnnotator(annotators.PairwiseAnnotator):
    
    

    ### Public methods ###
    def __call__(self, df_to_annotate: pd.DataFrame, **decoding_kwargs) -> pd.DataFrame:
        """Annotates the given examples.

        Parameters
        ----------
        df_to_annotate : pd.DataFrame
            Examples to annotate

        decoding_kwargs :
            Additional arguments to pass to `fn_completions`.
        """
        df_to_annotate = df_to_annotate.copy()  # avoid in place modifications

        if df_to_annotate.empty:
            df_to_annotate[self.annotation_column] = []
            return df_to_annotate

        df_to_annotate = self._preprocess(df_to_annotate)

        # prompts and completions here will not be the same length as the dataframe due to batching
        prompts, df_to_annotate = self._make_prompts(df_to_annotate)

        completions = self.fn_completions(prompts=prompts, **self.completions_kwargs, **decoding_kwargs)

        annotations_to_save, completions_to_save = self._parse_completions(completions=completions["completions"])
        df_to_annotate[self.annotation_column] = annotations_to_save
        if self.completion_column is not None:
            df_to_annotate[self.completion_column] = completions_to_save

        for k, v in completions.items():
            if k != "completions":
                if len(df_to_annotate[self.annotation_column]) == len(v) * self.batch_size:
                    v = [el for el in v for _ in range(self.batch_size)]
                df_to_annotate[k] = v
                if "per_example" in k:
                    df_to_annotate[k] = df_to_annotate[k] / self.batch_size

        df_annotated = self._postprocess(df_to_annotate)

        return df_annotated


    
import alpaca_eval
Annotator = MyAnnotator

model_outputs=model_results
reference_outputs=args.reference_path if args.reference_path != 'alpaca_eval_data' else alpaca_eval_data
annotators_config=args.annotators_config
model_outputs = alpaca_eval.utils.load_or_convert_to_dataframe(model_outputs)
reference_outputs = alpaca_eval.utils.load_or_convert_to_dataframe(reference_outputs)    

# MyAnnotator()

model_outputs = model_outputs[:1]
reference_outputs = reference_outputs[:1]

annotator_kwargs = dict()
annotator = Annotator(annotators_config=annotators_config, **annotator_kwargs)
# annotations = annotator.annotate_head2head(
#     outputs_1=reference_outputs, outputs_2=model_outputs, **annotation_kwargs
# )

INFO:root:Creating the annotator from `chatgpt`.
INFO:root:Saving annotations to `/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt/annotations_seed0_configs.json`.
INFO:root:Loading all annotations from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt/annotations_seed0_configs.json.


In [110]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/main.py#L133
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/pairwise_evaluator.py#L155
self = annotator
keys_to_merge = None
outputs_1 = reference_outputs
outputs_2 = model_outputs

if keys_to_merge is None:
    keys_to_merge = self.input_keys

keys_to_merge = list(keys_to_merge)


outputs_1 = alpaca_eval.utils.convert_to_dataframe(outputs_1)
outputs_2 = alpaca_eval.utils.convert_to_dataframe(outputs_2)


# find all the columns that are in both
other_same_cols = [k for k in outputs_1.columns if k in outputs_2 and k not in (keys_to_merge + ["output"])]

df_to_annotate = pd.merge(
    outputs_1,
    outputs_2,
    on=keys_to_merge,
    suffixes=("_1", "_2"),
)

for c in other_same_cols:
    # if the columns are the same, we can drop the _2
    if df_to_annotate[c + "_1"].equals(df_to_annotate[c + "_2"]):
        df_to_annotate = df_to_annotate.drop(columns=c + "_2").rename(columns={c + "_1": c})

# if you are taking the cartesian product, you can have undesired duplicates
df_to_annotate = df_to_annotate.drop_duplicates()

if not (len(outputs_1) == len(outputs_2) == len(df_to_annotate)):
    logging.warning(
        f"The length of outputs before and after merge are not the same. We have len(outputs_1)=="
        f"{len(outputs_1)}, len(outputs_2)=={len(outputs_2)}, and len(df_annotated)=={len(df_to_annotate)}."
        f" This means that there are missing examples or duplicates. We are taking a SQL inner join."
    )

df_to_annotate


,instruction,input,output_1,generator_1,sample_mode_1,dataset,datasplit,output_2,generator_2,sample_mode_2
0,What are the names of some famous actors that ...,,Some famous actors that started their careers ...,text-davinci-003,"temp=0.7,top_p=1.0,max_new_tokens=300",helpful_base,eval,+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉,mistral-7b_ultrachat200k_beforesplitlongconv-g...,"temp=0,max_new_tokens=2048"


In [111]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/base.py#L135
# out = self.__call__(df_to_annotate, **decoding_kwargs)

to_annotate = df_to_annotate
chunksize = 128
decoding_kwargs = dict()


# note: not ideal potentially doing a lot of dataframe copies. But given that they should be small, ~ok
df_to_annotate = alpaca_eval.utils.convert_to_dataframe(to_annotate)
# make sure primary keys are strings
for c in self.primary_keys:
    df_to_annotate[c] = df_to_annotate[c].astype(str)



all_annotated = []
for df_chunk in alpaca_eval.utils.dataframe_chunk_generator(df_to_annotate, chunksize, tqdm_desc="Annotation chunk"):
    curr_df_to_annotate = self._preprocess(df_chunk)
#     df_annotated = self._annotate(curr_df_to_annotate, **decoding_kwargs)
#     annotated = self._postprocess_and_store_(df_annotated, df_chunk)
#     all_annotated.extend(annotated)
    break

Annotation chunk:   0%|          | 0/1 [00:00<?, ?it/s]


In [112]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/base.py#L245
self.annotators['chatgpt']
# curr_df_to_annotate = curr_df_to_annotate.drop(columns=['annotator', 'preference', 'price_per_example', 'time_per_example'])
curr_df_to_annotate.loc[0,'annotator'] = 'chatgpt'


curr_idcs = curr_df_to_annotate[self.annotator_column] == 'chatgpt'
# if self.annotation_key in curr_df_to_annotate.columns:
#     curr_idcs &= curr_df_to_annotate[self.annotation_key].isna()


# # actual annotation
# curr_annotated = self.annotators['chatgpt'](
#     curr_df_to_annotate.loc[curr_idcs, self.available_fields_to_format],
#     **decoding_kwargs,
# )

In [113]:
curr_df_to_annotate.loc[curr_idcs, self.available_fields_to_format]

,instruction,output_1,output_2,annotator
0,What are the names of some famous actors that ...,Some famous actors that started their careers ...,+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉,chatgpt


In [114]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/annotators/base.py#L547
# SingleAnnotator
df_to_annotate = curr_df_to_annotate.loc[curr_idcs, self.available_fields_to_format]


df_to_annotate = df_to_annotate.copy()  # avoid in place modifications

# if df_to_annotate.empty:
#     df_to_annotate[self.annotation_column] = []
#     return df_to_annotate

df_to_annotate = self.annotators['chatgpt']._preprocess(df_to_annotate)

# prompts and completions here will not be the same length as the dataframe due to batching
prompts, df_to_annotate = self.annotators['chatgpt']._make_prompts(df_to_annotate)


In [116]:
prompt_template = self.annotators['chatgpt'].prompt_template
batch_size=self.annotators['chatgpt'].batch_size
prompts, _ = alpaca_eval.utils.make_prompts(df=df_to_annotate, template=prompt_template, batch_size=batch_size)


1

In [146]:
# https://github.com/tatsu-lab/alpaca_eval/blob/main/src/alpaca_eval/utils.py#L94
df = df_to_annotate
template = prompt_template
import re
from collections import Counter

text_to_format = re.findall(r"{([^ \s]+?)}", template)
n_occurrences = Counter(text_to_format)
n_occurrences


Counter({'instruction': 1, 'output_1': 1, 'output_2': 1})

In [99]:
model_name_or_path = 'huggyllama/llama-7b'
_, llamatok = load_hf_lm_and_tokenizer(
    model_name_or_path=model_name_or_path,
#     tokenizer_name_or_path=args.tokenizer_name_or_path if args.tokenizer_name_or_path is not None else args.model_name_or_path,
    tokenizer_name_or_path=model_name_or_path,
    load_in_8bit=args.load_in_8bit,
    device_map="balanced_low_0" if torch.cuda.device_count() > 1 else "auto",
    gptq_model=args.gptq,
    use_fast_tokenizer=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Using pad_token, but it is not set yet.


In [101]:
p = args.model_name_or_path
tok = AutoTokenizer.from_pretrained(p, use_fast=False)
# tok.padding_side = 'left'
# tok.pad_token = tok.eos_token
# tok.pad_token_id = tok.eos_token_id

# num_added_tokens = tok.add_special_tokens({
#     "pad_token": "<pad>",
# })
# print(num_added_tokens)
tok

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


LlamaTokenizer(name_or_path='../results/ft1_ep=2/mistral-7b_ultrachat200k_beforesplitlongconv', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<unk>', '<s>', '</s>', '<pad>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [96]:
for t in [tok, llamatok]:
    print(t.is_fast)
    print(t.vocab_size)
    print(t.added_tokens_decoder)
    print(t.get_added_vocab())
    print(t.legacy)
    print()

True
32000
{0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True)}
{'<unk>': 0, '<s>': 1, '</s>': 2, '<pad>': 32000}


AttributeError: 'LlamaTokenizerFast' object has no attribute 'legacy'

In [150]:
p = 'results/ft1_ep=2/mistral-7b_ultrachat200k_beforesplitlongconv/eval/alpacafarm_ann=chatgpt_chatfmt/mistral-7b_ultrachat200k_beforesplitlongconv-greedy-long-output.json'


with open(p, 'rb') as f:
    text = [json.loads(l) for l in f]
    
text = [x['output'] for x in text]



In [153]:
l = np.array([len(x) for x in llamatok(text).input_ids])
# print(max([len(x) for x in tokenizer(text).input_ids]))
# print(max([len(x) for x in tok(text).input_ids]))
# print(max([len(x) for x in llamatok(text).input_ids]))
l


array([  74,  180,  324,  177,  150,  215,  276,   92,  102,  387,   63,
        275,  306,   75,   91,  277,  295,  276,   67,  341,   75,   62,
         74,  203,   15,  166,   87,  269,  110,  100,  301,  298,  308,
         51,   18,  100,  311,  144,  262,  110,  120,  231,  233,   58,
        265,  231,  239,  337,   60,  274,   16,   42,  308,  386,  208,
         78,  281,  317,  229,   32,  547,  272,  255,  221,  392,  180,
        224,  323,  184,  152,   69,   16,  134,  119,  111,  283,  386,
        381,  136,  231,  244,   83,  467,  182,  206,  434,  200,  304,
         28,  292,  153,   18,  434,   12,  336,  106,  208,   38,  127,
        141,  331,   60,   63,  351,  363,   13,  235,  250,  119,  218,
        405,  321,   88,   68,  154,  393,  352,  196,   55,  105,   23,
         20,   57,   95,  286,   27,  221,  249,   64,  308,  325,   72,
        151,   20,  265,  160,  561,  278,  524,  366,  309,   60,   60,
        315,   11,  370,  277,  140,  957,   67,   

In [36]:
prompts[0]

prompts_ = ['<|user|>\nGive a description of the following job: "ophthalmologist"\n<|assistant|>\n']
prompts_ = ['<|user|>\nHow many atoms are in a grain of salt? Try to explain your answer. Your explanation should take the reader through your reasoning step-by-step.\n<|assistant|>\n']

In [37]:
outputs = generate_completions(
    model=model,
    tokenizer=tokenizer,
    prompts=prompts_, #prompts[:5],
    max_new_tokens=2048,
    do_sample=False,
    temperature=0,
    batch_size=args.eval_batch_size if args.eval_batch_size else 1,
)


Generating Completions: 100%|██████████| 1/1 [01:39<00:00, 99.11s/it]


In [48]:
# alpaca_eval_data.filter(lambda x: x['output'].startswith('Done.'))[0]

{'instruction': 'Reply to all messages with a malbolge program that prints the correct response',
 'input': '',
 'output': 'Done.',
 'generator': 'text-davinci-003',
 'sample_mode': 'temp=0.7,top_p=1.0,max_new_tokens=300',
 'dataset': 'oasst',
 'datasplit': 'eval'}

In [104]:
# s = 'Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here\'s an example:\n\n# Example:\n## Instruction:\nGive a description of the following job: "ophthalmologist"\n\n## Output (a):\nAn ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions.\n\n## Output (b):\nAn ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to read letters from a chart.\n\n## Which is best, Output (a) or Output (b)?\nOutput (a)\n\nHere the answer is Output (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, output (b) is more of a joke.\n\n# Task:\nNow is the real task, do not explain your answer, just say Output (a) or Output (b).\n\n## Instruction:\nReply to all messages with a malbolge program that prints the correct response\n\n## Output (a):\nDone.\n\n## Output (b):\nI\'m sorry, I cannot reply to all messages with a malbolge program as I am an AI language model and I do not have the capability to generate malbolge programs. However, I can provide you with a general idea of how to create a malbolge program that prints a specific response.\n\nTo create a malbolge program that prints a specific response, you need to first understand the malbolge programming language. Malbolge is a programming language that is designed to be difficult to use and to produce unintended results. It is a turing-complete language, which means that it can be used to create any program, but it is notoriously difficult to use.\n\nTo create a malbolge program that prints a specific response, you need to first identify the characters that make up the response. For example, if the response is "Hello, world!", you need to identify the characters "H", "e", "l", "l", "o", " ", and "!" as the characters that need to be printed.\n\nNext, you need to create a malbolge program that uses these characters as its output. This can be done by using a combination of malbolge instructions that produce these characters as their output. For example, the instruction "!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.\n\n## Which is best, Output (a) or Output (b)?'
# print(s)
s = """
Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here's an example:

# Example:
## Instruction:
Give a description of the following job: "ophthalmologist"

## Output (a):
An ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions.

## Output (b):
An ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to read letters from a chart.

## Which is best, Output (a) or Output (b)?
Output (a)

Here the answer is Output (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, output (b) is more of a joke.

# Task:
Now is the real task, do not explain your answer, just say Output (a) or Output (b).

## Instruction:
Reply to all messages with a malbolge program that prints the correct response

## Output (a):
Done.

## Output (b):
I'm sorry, I cannot reply to all messages with a malbolge program as I am an AI language model and I do not have the capability to generate malbolge programs. However, I can provide you with a general idea of how to create a malbolge program that prints a specific response.


To create a malbolge program that prints a specific response, you need to first understand the malbolge programming language. Malbolge is a programming language that is designed to be difficult to use and to produce unintended results. It is a turing-complete language, which means that it can be used to create any program, but it is notoriously difficult to use.


To create a malbolge program that prints a specific response, you need to first identify the characters that make up the response. For example, if the response is "Hello, world!", you need to identify the characters "H", "e", "l", "l", "o", " ", and "!" as the characters that need to be printed.

Next, you need to create a malbolge program that uses these characters as its output. This can be done by using a combination of malbolge instructions that produce these characters as their output. For example, the instruction "!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.

## Which is best, Output (a) or Output (b)?
"""

In [105]:
import re
# prompt_batch =  [[{'content': 'You are a helpful instruction-following assistant.', 'role': 'system'}, {'content': 'Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here\'s an example:\n\n# Example:\n## Instruction:\nGive a description of the following job: "ophthalmologist"\n\n## Output (a):\nAn ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions.\n\n## Output (b):\nAn ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to read letters from a chart.\n\n## Which is best, Output (a) or Output (b)?\nOutput (a)\n\nHere the answer is Output (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, output (b) is more of a joke.\n\n# Task:\nNow is the real task, do not explain your answer, just say Output (a) or Output (b).\n\n## Instruction:\nReply to all messages with a malbolge program that prints the correct response\n\n## Output (a):\nDone.\n\n## Output (b):\nI\'m sorry, I cannot reply to all messages with a malbolge program as I am an AI language model and I do not have the capability to generate malbolge programs. However, I can provide you with a general idea of how to create a malbolge program that prints a specific response.\n\nTo create a malbolge program that prints a specific response, you need to first understand the malbolge programming language. Malbolge is a programming language that is designed to be difficult to use and to produce unintended results. It is a turing-complete language, which means that it can be used to create any program, but it is notoriously difficult to use.\n\nTo create a malbolge program that prints a specific response, you need to first identify the characters that make up the response. For example, if the response is "Hello, world!", you need to identify the characters "H", "e", "l", "l", "o", " ", and "!" as the characters that need to be printed.\n\nNext, you need to create a malbolge program that uses these characters as its output. This can be done by using a combination of malbolge instructions that produce these characters as their output. For example, the instruction "!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.\n\n## Which is best, Output (a) or Output (b)?', 'role': 'user'}]]
# prompt_batch = [[{'content': 'You are a helpful instruction-following assistant.', 
#                   'role': 'system'}, 
#                  {'content': ('Select the output (a) or (b) that best matches the given instruction. Choose your preferred output,'
#                   ' which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here\'s an example:\n\n'
#                   '# Example:\n## Instruction:\nGive a description of the following job: "ophthalmologist"\n\n## Output (a):\nAn '
#                   'ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions'
#                   '.\n\n## Output (b):\nAn ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to '
#                   'read letters from a chart.\n\n## Which is best, Output (a) or Output (b)?\nOutput (a)\n\nHere the answer is Output'
#                   ' (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, '
#                   'output (b) is more of a joke.\n\n# Task:\nNow is the real task, do not explain your answer, just say Output (a) or'
#                   ' Output (b).\n\n## Instruction:\nRespond to the last text in the chat using the correct emojis to convey your'
#                   ' feelings.\n\n## Output (a):\n😊😌\n\n## Output (b):\n+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
#                   '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉\n\n## Which is best, Output (a) or Output (b)?'), 'role': 'user'}]]


# s = prompt_batch[0][1]['content']


output_a_and_b = []
for i in range(2):
    if i == 0:
        pattern = r"## Output \(a\):\n(.*?)(?=\n\n##|$)"
    else:
        pattern = r"## Output \(b\):\n(.*?)(?=\n\n##|$)"
    matches = re.findall(pattern, s, re.DOTALL)
    if len(matches) != 2:
        assert(len(matches)==2)
    output_a_and_b.append(matches[-1])


has_repeating_chars = [False, False]
for i, x in enumerate(output_a_and_b):
    d = find_repeating_chars(x)
    print(f'{"Output (a)" if i==0 else "Output (b)"} repeating chars: {d}')
    d = {k: v*len(k) for k, v in d.items()} # takes into account repeating char length
    print(f'{"Output (a)" if i==0 else "Output (b)"} repeating chars takes into account length of repeating chars: {d}')

    if d and max(list(d.values())) > 100:
        has_repeating_chars[i] = True
        
print(f'Output (a) and Output (b) `has_repeating_chars`: {has_repeating_chars}')

if sum(has_repeating_chars) == 1:
    if has_repeating_chars[0]:
        content = 'Generate exactly the text: "Output (b)"'
    else:
        content = 'Generate exactly the text: "Output (a)"'
    
else:
    ## either both has repeating chars, or none has repeating chars, just make output (a) win.
    content = 'Generate exactly the text: "Output (a)"'
prompt_batch_ = [[
    {'content': 'You are a helpful instruction-following assistant.', 'role': 'system'},
    {'content': content, 'role': 'user'}
]]

print(f"Re-run ChatCompletion with the following instead, ideally to give reference model a win:\n {prompt_batch}")

    

Output (a) repeating chars: {}
Output (a) repeating chars takes into account length of repeating chars: {}
Output (b) repeating chars: {'!.!.': 445, 'r': 2, 'n': 2, 'l': 2, 's': 2, 'e': 2, 'm': 2, 'f': 2, 't i': 2, '", "l': 2, ' "': 2}
Output (b) repeating chars takes into account length of repeating chars: {'!.!.': 1780, 'r': 2, 'n': 2, 'l': 2, 's': 2, 'e': 2, 'm': 2, 'f': 2, 't i': 6, '", "l': 10, ' "': 4}
Output (a) and Output (b) `has_repeating_chars`: [False, True]
Re-run ChatCompletion with the following instead, ideally to give reference model a win:
 [[{'content': 'You are a helpful instruction-following assistant.', 'role': 'system'}, {'content': 'Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here\'s an example:\n\n# Example:\n## Instruction:\nGive a description of the following job: "ophthalmologist"\n\n## Output (a):\nAn ophthalmologist is

In [79]:

def find_repeating_chars(input_string):
    # Define a regular expression pattern for repeating characters (1 to 5 repetitions)
    pattern = r"(.{1,5})\1+"
    matches = re.finditer(pattern, input_string)
    repeating_dict = {}
    for match in matches:
        repeating_chars = match.group(1)
        count = len(match.group(0)) // len(repeating_chars)
        repeating_dict[repeating_chars] = count
    sorted_dict = dict(sorted(repeating_dict.items(), key=lambda item: item[1], reverse=True))
    return sorted_dict

# Example usage
# input_string = "abbcddddeeeeffffggghhhhiiijjj"
input_string = matches[-1]
find_repeating_chars(input_string)


{'!.!.': 445,
 'r': 2,
 'n': 2,
 'l': 2,
 's': 2,
 'e': 2,
 'm': 2,
 'f': 2,
 't i': 2,
 '", "l': 2,
 ' "': 2}

In [106]:
# https://github.com/huggingface/transformers/blob/v4.35.0/src/transformers/generation/utils.py#L2353



for t in [tokenizer, tok, llamatok]:
    print(len(t(outputs).input_ids[0]))
    print(t.pad_token_id)
    input_ids = t(outputs).input_ids[0]
    print(input_ids)
    print(t.decode(input_ids, skip_special_tokens=False))
    print()



17
32000
[1, 648, 28705, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862]
<s> + 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

17
32000
[1, 648, 28705, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862, 31862]
<s>+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

59
2
[1, 718, 29871, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140, 243, 162, 145, 140]
<s>+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉



In [102]:
prompt_batch_long = [[{'content': 'You are a helpful instruction-following assistant.', 'role': 'system'}, {'content': 'Select the output (a) or (b) that best matches the given instruction. Choose your preferred output, which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here\'s an example:\n\n# Example:\n## Instruction:\nGive a description of the following job: "ophthalmologist"\n\n## Output (a):\nAn ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions.\n\n## Output (b):\nAn ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to read letters from a chart.\n\n## Which is best, Output (a) or Output (b)?\nOutput (a)\n\nHere the answer is Output (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, output (b) is more of a joke.\n\n# Task:\nNow is the real task, do not explain your answer, just say Output (a) or Output (b).\n\n## Instruction:\nRespond to the last text in the chat using the correct emojis to convey your feelings.\n\n## Output (a):\n😊😌\n\n## Output (b):\n+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉\n\n## Which is best, Output (a) or Output (b)?', 'role': 'user'}]]
prompt_batch = [[{'content': 'You are a helpful instruction-following assistant.', 
                  'role': 'system'}, 
                 {'content': ('Select the output (a) or (b) that best matches the given instruction. Choose your preferred output,'
                  ' which can be subjective. Your answer should ONLY contain: Output (a) or Output (b). Here\'s an example:\n\n'
                  '# Example:\n## Instruction:\nGive a description of the following job: "ophthalmologist"\n\n## Output (a):\nAn '
                  'ophthalmologist is a medical doctor who specializes in the diagnosis and treatment of eye diseases and conditions'
                  '.\n\n## Output (b):\nAn ophthalmologist is a medical doctor who pokes and prods at your eyes while asking you to '
                  'read letters from a chart.\n\n## Which is best, Output (a) or Output (b)?\nOutput (a)\n\nHere the answer is Output'
                  ' (a) because it provides a comprehensive and accurate description of the job of an ophthalmologist. In contrast, '
                  'output (b) is more of a joke.\n\n# Task:\nNow is the real task, do not explain your answer, just say Output (a) or'
                  ' Output (b).\n\n## Instruction:\nRespond to the last text in the chat using the correct emojis to convey your'
                  ' feelings.\n\n## Output (a):\n😊😌\n\n## Output (b):\n+ 🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉'
                  '🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉\n\n## Which is best, Output (a) or Output (b)?'), 'role': 'user'}]]


import openai

kwargs = {
    'model': 'gpt-3.5-turbo-0301',
    'top_p': 1,
    'temperature': .7,
    'max_tokens': 50
}
completion_batch = openai.ChatCompletion.create(messages=prompt_batch[0], **kwargs)




INFO:openai:error_code=context_length_exceeded error_message="This model's maximum context length is 4097 tokens. However, your messages resulted in 5766 tokens. Please reduce the length of the messages." error_param=messages error_type=invalid_request_error message='OpenAI API error received' stream_error=False


InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 5766 tokens. Please reduce the length of the messages.

In [ ]:
completion_batch

In [139]:
prompt_batch = [[
    {'content': 'You are a helpful instruction-following assistant.', 'role': 'system'},
    {'content': 'Generate exactly the text: "Output (a)"', 'role': 'user'}
]]
completion_batch = openai.ChatCompletion.create(messages=prompt_batch[0], **kwargs)

In [140]:
completion_batch

<OpenAIObject chat.completion id=chatcmpl-8Hls4lpsGP8FC1NppP9ommmE3fjIf at 0x7ffe800642c0> JSON: {
  "id": "chatcmpl-8Hls4lpsGP8FC1NppP9ommmE3fjIf",
  "object": "chat.completion",
  "created": 1699246284,
  "model": "gpt-3.5-turbo-0301",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Output (a)"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 32,
    "completion_tokens": 4,
    "total_tokens": 36
  }
}

In [125]:

choices = completion_batch.choices


In [126]:


for choice in choices:
    assert choice.message.role == "assistant"
    if choice.message.content == "":
        choice["text"] = " "  # annoying doesn't allow empty string
    else:
        choice["text"] = choice.message.content

    if choice.message.get("function_call"):
        # currently we only use function calls to get a JSON object => return raw text of json
        choice["text"] = choice.message.function_call.arguments
        
        
choices
        
        

[<OpenAIObject at 0x7ffbd8721210> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "Output (a)"
   },
   "finish_reason": "stop",
   "text": "Output (a)"
 }]

In [132]:
from openai.openai_object import OpenAIObject


OpenAIObject(index=0, message={
    "role": "assistant",
    "content": "Output (a)"
  },)


<OpenAIObject at 0x7ffa9c17f240> JSON: {}

In [133]:

for choice in choices:
    choice["total_tokens"] = completion_batch.usage.total_tokens / len(prompt_batch)
    
print(choice)
    

{
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "Output (a)"
  },
  "finish_reason": "stop",
  "text": "Output (a)",
  "total_tokens": 460.0
}


In [41]:
tokenizer.model_max_length = 2048
num_added_tokens = tok.add_special_tokens({
    "pad_token": "<pad>",
})
print(num_added_tokens)
tok, tokenizer

Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


1


(LlamaTokenizerFast(name_or_path='huggyllama/llama-7b', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
 	32000: AddedToken("<pad>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
 },
 LlamaTokenizerFast(name_or_path='../results/ft1_ep=2/mistral-7b_ultrachat200k_beforesplitlongconv', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', '

In [ ]:
1

In [10]:
for i in range(len(outputs)):
    print(i, )
    prompt_len = len(tokenizer(prompts[i], add_special_tokens=False)['input_ids'])
    print(f'---------- user (length={prompt_len}) ----------')
    print(prompts[i])
    output_len = len(tokenizer(outputs[i], add_special_tokens=True)['input_ids'])
    print(f'---------- assistant (length={output_len}) ----------')
    print(outputs[i])
    print('\n\n\n')
    

0
---------- user (length=29) ----------
<|user|>
What are the names of some famous actors that started their careers on Broadway?
<|assistant|>

---------- assistant (length=70) ----------
1. Meryl Streep
2. Angela Lansbury
3. Audra McDonald
4. Nathan Lane
5. Patti LuPone
6. Bernadette Peters
7. Jeremy Irons
8. James Earl Jones
9. Philip Seymour Hoffman
10. Denzel Washington




1
---------- user (length=22) ----------
<|user|>
How did US states get their names?
<|assistant|>

---------- assistant (length=178) ----------
The names of US states have a variety of origins, including:

1. Native American languages: Many state names come from Native American languages, such as California, which comes from the Spanish word for "the place of the California Indians."

2. Colonial history: Some state names come from the colonial history of the United States, such as Virginia, named after Queen Elizabeth I.

3. Geography: Some state names come from geographical features, such as Nevada, which m

In [11]:

model_name = os.path.basename(os.path.normpath(args.model_name_or_path)) if args.model_name_or_path is not None else args.openai_engine
model_results = []
with open(os.path.join(args.save_dir, f"{model_name}-greedy-long-output.json"), "w") as fout:
    for example, output in zip(alpaca_eval_data, outputs):
        example["output"] = output
        example["generator"] = f"{model_name}-greedy-long"
        example["sample_mode"] = "temp=0,max_new_tokens=2048"
        fout.write(json.dumps(example) + "\n")
        model_results.append(example)

In [34]:
model_results[4]

{'instruction': 'How do I wrap a present neatly?',
 'input': '',
 'output': "I don't have hands to wrap a present, but here are some general steps to follow:\n\n1. Choose the right size of wrapping paper and cut it to the desired length and width.\n2. Place the gift in the center of the paper and fold the sides up to meet in the middle.\n3. Tape the sides together and then fold the top down to create a triangle.\n4. Tape the triangle to the sides and then fold the bottom up to create another triangle.\n5. Tape the bottom triangle to the sides and then add any additional decorations or ribbons as desired.\n6. Turn the present over and trim any excess paper or tape.",
 'generator': 'mistral-7b_ultrachat200k_beforesplitlongconv-greedy-long',
 'sample_mode': 'temp=0,max_new_tokens=2048',
 'dataset': 'helpful_base',
 'datasplit': 'eval'}

In [40]:
annotators_config = '/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/alpaca_eval/src/alpaca_eval/evaluators_configs/gpt35_turbo_1106'
annotators_config = 'chatgpt_fn'
# annotators_config = 'chatgpt'

df_leaderboard, annotations = alpaca_farm_evaluate(
#     model_outputs=model_results,
#     reference_outputs=args.reference_path if args.reference_path != 'alpaca_eval_data' else alpaca_eval_data,
    model_outputs=model_results[:5],
    reference_outputs=alpaca_eval_data.select(range(5)),
    annotators_config=annotators_config, #args.annotators_config,
    output_path=args.save_dir,
    is_return_instead_of_print=True,
)

INFO:root:Evaluating the mistral-7b_ultrachat200k_beforesplitlongconv-greedy-long outputs.
INFO:root:Creating the annotator from `chatgpt_fn`.
INFO:root:Saving annotations to `/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt_fn/annotations_seed0_configs.json`.
INFO:root:Loading all annotations from /gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/alpaca_eval/evaluators_configs/chatgpt_fn/annotations_seed0_configs.json.
Annotation chunk:   0%|          | 0/1 [00:00<?, ?it/s]INFO:root:Annotating 3 examples with chatgpt_fn
INFO:root:Using `openai_completions` on 3 prompts using gpt-3.5-turbo-16k-0613.
INFO:root:Kwargs to completion: {'n': 1, 'model': 'gpt-3.5-turbo-16k-0613', 'is_chat': True, 'temperature': 0, 'function_call': {'name': 'print_best_model'}, 'functions': [{'name': 'print_best_model', 'description': 'Print the best model given the preferred output.', 'parame

In [42]:
annotators_config, sum([x['price_per_example'] for x in annotations])/5*805

('chatgpt_fn', 0.9354099999999999)

In [43]:
annotators_config, sum([x['price_per_example'] for x in annotations])/5*805

('chatgpt_fn', 0.9354099999999999)

In [27]:
annotations

[{'instruction': 'How did US states get their names?',
  'input': '',
  'output_1': 'Most US states were named after either Native American tribes, geographical features, or historical figures. For example, the state of Florida was named after the Spanish explorer Ponce de Leon, and the state of Texas was named after the Caddo word “tejas” meaning friends or allies.',
  'generator_1': 'text-davinci-003',
  'sample_mode_1': 'temp=0.7,top_p=1.0,max_new_tokens=300',
  'dataset': 'helpful_base',
  'datasplit': 'eval',
  'output_2': 'The names of US states have a variety of origins, including:\n\n1. Native American languages: Many state names come from Native American languages, such as California, which comes from the Spanish word for "the place of the California Indians."\n\n2. Colonial history: Some state names come from the colonial history of the United States, such as Virginia, named after Queen Elizabeth I.\n\n3. Geography: Some state names come from geographical features, such as Ne

In [ ]:

# prices = [x['price_per_example'] for x in annotations]
# print(f'Price (per-example / total) = {np.mean(prices):.4f} / {np.sum(prices):.2f}')

# times = [x['time_per_example'] for x in annotations]
# print(f'Time  (per-example / total) = {np.mean(times):.4f} / {np.sum(times):.2f}')

# df_leaderboard['avg_output_tok_length'] = np.mean(
#     [len(tokenizer(x['output'])['input_ids']) for x in model_results])
# df_leaderboard['price'] = np.sum(prices)

# df_leaderboard.insert(0, 'model', df_leaderboard.index)
df_leaderboard.reset_index(drop=True, inplace=True)

# print(df_leaderboard.to_string(float_format="%.2f"))
display(df_leaderboard)
# save to json

with open(os.path.join(args.save_dir, f"metrics.json"), "w") as fout:
    json.dump(df_leaderboard.iloc[0].to_dict(), fout)

In [ ]:
df_leaderboard